#To run the program please run the last cell which will cal all the functions but before running that cell please #run each and every cell other than to compile the functions that will be used to run the model.



In [ ]:
#holds code for importing libraries

import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

from keras.layers import Input
from keras.models import Model, Sequential
from keras.layers.core import Reshape, Dense, Dropout, Flatten
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Convolution2D, UpSampling2D
from keras.datasets import mnist
from keras.optimizers import Adam
from keras import backend as K
from keras import initializers

In [ ]:
#holds code for loading MNIST data and creating test/train split

np.random.seed(1000)
randomDim = 10

# Load MNIST data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = (X_train.astype(np.float32) - 127.5)/127.5
X_train = X_train.reshape(60000, 784)

In [ ]:
#optimizer
adam = Adam(lr=0.0002, beta_1=0.5)

In [ ]:
#Discrimnator model

discriminator = Sequential()
discriminator.add(Dense(1024, input_dim=784, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))
discriminator.add(Dense(512))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))
discriminator.add(Dense(256))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.compile(loss='binary_crossentropy', optimizer=adam)

In [ ]:
#Generator Model
generator = Sequential()
generator.add(Dense(256, input_dim=randomDim, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
generator.add(LeakyReLU(0.2))
generator.add(Dense(512))
generator.add(LeakyReLU(0.2))
generator.add(Dense(1024))
generator.add(LeakyReLU(0.2))
generator.add(Dense(784, activation='tanh'))
generator.compile(loss='binary_crossentropy', optimizer=adam)

In [ ]:
#function for training model

def train(epochs=1, batchSize=128):
    batchCount = round(X_train.shape[0] / batchSize)
    

    for e in range(1, epochs+1):
        for _ in tqdm(range(batchCount)):
            noise = np.random.normal(0, 1, size=[batchSize, randomDim])
            imageBatch = X_train[np.random.randint(0, X_train.shape[0], size=batchSize)]
            generatedImages = generator.predict(noise)
            X = np.concatenate([imageBatch, generatedImages])
            yDis = np.zeros(2*batchSize)
            yDis[:batchSize] = 0.9

            # This part trains the discriminator
            discriminator.trainable = True
            dloss = discriminator.train_on_batch(X, yDis)

            # This part trains the generator
            noise = np.random.normal(0, 1, size=[batchSize, randomDim])
            yGen = np.ones(batchSize)
            discriminator.trainable = False
            gloss = gan.train_on_batch(noise, yGen)

        dLosses.append(dloss)
        gLosses.append(gloss)

        if e == 1 or e % 20 == 0:
            plotGeneratedImages(e)
            saveModels(e)
    plotLoss(e)

In [ ]:
# Combined network
discriminator.trainable = False
ganInput = Input(shape=(randomDim,))
x = generator(ganInput)
ganOutput = discriminator(x)
gan = Model(inputs=ganInput, outputs=ganOutput)
gan.compile(loss='binary_crossentropy', optimizer=adam)

In [ ]:
# Plotting Losses

dLosses = []
gLosses = []

def plotLoss(epoch):
    plt.figure(figsize=(10, 8))
    plt.plot(dLosses, label='Discriminitive loss')
    plt.plot(gLosses, label='Generative loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig('/Users/gauravthapliyal/Fall 2020/Machine Learning/ML Project/GAN_images/gan_loss_epoch_%d.png' % epoch)

In [ ]:
#Function for saving models and image generation

def saveModels(epoch):
    generator.save('/Users/gauravthapliyal/Fall 2020/Machine Learning/ML Project/models/gan_generator_epoch_%d.h5' % epoch)
    discriminator.save('/Users/gauravthapliyal/Fall 2020/Machine Learning/ML Project/models/gan_discriminator_epoch_%d.h5' % epoch)


def plotGeneratedImages(epoch, examples=100, dim=(10, 10), figsize=(10, 10)):
    noise = np.random.normal(0, 1, size=[examples, randomDim])
    generatedImages = generator.predict(noise)
    generatedImages = generatedImages.reshape(examples, 28, 28)

    plt.figure(figsize=figsize)
    for i in range(generatedImages.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generatedImages[i], interpolation='nearest', cmap='gray_r')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('/Users/gauravthapliyal/Fall 2020/Machine Learning/ML Project/GAN_images/gan_generated_image_epoch_%d.png' % epoch)




In [ ]:
#run the model

if __name__ == '__main__':
    train(1, 128)